In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time
from datetime import date
import csv

In [5]:
# access the url and pull its html

# Set up Selenium
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
#options.add_argument("--headless")  # Run in headless mode
driver = webdriver.Chrome(service=service, options=options)

url = "https://www.amazon.com/s?k=SQL&i=stripbooks-intl-ship&rh=n%3A283155%2Cp_n_publication_date%3A1250226011%2Cp_n_feature_twenty-five_browse-bin%3A3291437011&dc&crid=3K0GREPYTP9TO&qid=1742274102&rnid=3291435011&sprefix=sql%2Cstripbooks-intl-ship%2C349&ref=sr_nr_p_n_feature_twenty-five_browse-bin_1&ds=v1%3Adu8kmx4MO74g6tVdHoRUhwwUAb%2B8qUqJzO%2FIspmDU9U"
#url = "https://www.amazon.com/s?k=SQL&i=stripbooks-intl-ship&rh=n%3A283155%2Cp_n_publication_date%3A1250226011%2Cp_n_feature_nine_browse-bin%3A3291437011&dc&qid=1735623011&rnid=3291435011&xpid=HDtZxkKRxYd5i&ref=sr_pg_1"
driver.get(url)
driver.implicitely_wait(10)


ConnectionError: Could not reach host. Are you offline?

In [22]:
# Iterate over the item list and scrapp the needed informations

Book_titles = []
Book_authors = []
Dates_published = []

Kindle_prices = []
Paperback_prices = []
Hardcover_prices = []

Date_extracted = []

def scrape_page():

    driver.current_url
    soup = BeautifulSoup(driver.page_source, "html.parser")

    Item_List = soup.find_all('div', {'role':'listitem'})
    print(len(Item_List))
   
    for i in Item_List:
    
        Price_type_lists = []
        Price_lists = []
    
        # Extract book title
        Book_title = i.find('h2', class_="a-size-medium a-spacing-none a-color-base a-text-normal")
        Book_title = Book_title.get_text(strip=True)
        Book_titles.append(Book_title if Book_title else "N/A")
    
        # Extract book author
        Book_author = i.find("a", class_="a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style")
    
        if not Book_author:
            container = i.find("div", class_="a-row")
            if container:
                # Look for the author specifically within "by" context
                text_parts = container.get_text(strip=True).split("|")
                for part in text_parts:
                    if "by" in part.lower():
                        # Extract the author's name and clean up the text
                        Book_author = part.split("by")[1].strip()
                        break
    
        # Append the author name, ensuring no publication date is included
        if hasattr(Book_author, "get_text"):
            Book_authors.append(Book_author.get_text(strip=True))  # Extract the text from the tag
    
        elif Book_author:
            Book_authors.append(Book_author)  # Use the extracted text directly
        else:
            Book_authors.append("N/A")  # Default value if no author is found
    
        
        # Extract published date
        Date_published = i.find("span", class_="a-size-base a-color-secondary a-text-normal")
        Date_published = Date_published.get_text(strip=True)
        Dates_published.append(Date_published if Date_published else "N/A")
    
        # Extract kindle price
    
        Index_price_type_lists = i.find_all("a", class_="a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style a-text-bold")
        Index_price_lists = i.find_all("span", class_="a-offscreen")
    
        for j in Index_price_type_lists:
            Price_type_lists.append(j.get_text(strip=True))
    
        for k in Index_price_lists:
            Price_lists.append(k.get_text(strip=True))
    
        price_dict = dict(zip(Price_type_lists, Price_lists))
    
        # Extract kindle price
    
        search_term = "Kindle"
        matching_key = next((key for key in price_dict if search_term in key), None)
    
        # Assign the value to a variable
        if matching_key:
            Kindle_price = price_dict[matching_key]
            Kindle_prices.append(Kindle_price)
        
        else:
            Kindle_prices.append("N/A")
    
    
        # Extract Paperback price
    
        search_term = "Paperback"
        matching_key = next((key for key in price_dict if search_term in key), None)
    
        # Assign the value to a variable
        if matching_key:
            Paperback_price = price_dict[matching_key]
            Paperback_prices.append(Paperback_price)
        
        else:
            Paperback_prices.append("N/A")
            
        # Extract Harcover price
    
        search_term = "Hardcover"
        matching_key = next((key for key in price_dict if search_term in key), None)
    
        # Assign the value to a variable
        if matching_key:
            Hardcover_price = price_dict[matching_key]
            Hardcover_prices.append(Hardcover_price)
        
        else:
            Hardcover_prices.append("N/A")

        # write the date that the information is extracted 
        Date_extracted.append(date.today())
        

In [24]:
scrape_page()

16


In [26]:
# Function to navigate through pages

def go_to_next_page():
    try:
        # Find the "Next" button (adjust based on the actual button's identifier)
        next_button = driver.find_element(By.XPATH, "//a[text()='Next']")  # If it uses text 'Next'
        next_button.click()  # Click the "Next" button
        time.sleep(30)  # Wait for the page to load
    except Exception as e:
        print("No more pages or error:",e)
        return False
    return True

In [28]:
# Scrape the first page
scrape_page()

# Loop through and scrape data from subsequent pages

while go_to_next_page():
    scrape_page()  # Scrape data after clicking the "Next" button

# Once done, close the browser
driver.quit()

16
16
16
16
16
16
16
6
No more pages or error: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[text()='Next']"}
  (Session info: chrome=134.0.6998.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x005FB5A3+24387]
	(No symbol) [0x00585904]
	(No symbol) [0x00460753]
	(No symbol) [0x004A8BA9]
	(No symbol) [0x004A8EFB]
	(No symbol) [0x004F19C2]
	(No symbol) [0x004CD894]
	(No symbol) [0x004EF138]
	(No symbol) [0x004CD646]
	(No symbol) [0x0049C59F]
	(No symbol) [0x0049D8E4]
	GetHandleVerifier [0x008FD883+3179043]
	GetHandleVerifier [0x00916CF9+3282585]
	GetHandleVerifier [0x0091167C+3260444]
	GetHandleVerifier [0x00694330+650448]
	(No symbol) [0x0058ED0D]
	(No symbol) [0x0058BAF8]
	(No symbol) [0x0058BC99]
	(No symbol) [0x0057E530]
	BaseThreadInitThunk [0x75DE6839+25]
	RtlGetFullPathName_UEx [0x76EC906F+1215]
	RtlGetFullPa

In [30]:
import pandas as pd

df = pd.DataFrame({"Title":Book_titles,
                   "Author":Book_authors,
                   "Published_Date":Dates_published,
                   "Kindle Price":Kindle_prices,
                   "Paperback Price":Paperback_prices,
                   "Hardcover price":Hardcover_prices,
                   "Extraction Date":Date_extracted})
styled_df = df.style.set_properties(**{'text-align': 'left'})
styled_df = styled_df.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

In [32]:
styled_df

,Title,Author,Published_Date,Kindle Price,Paperback Price,Hardcover price,Extraction Date
0,"Python, Java, SQL, JavaScript & C++: The Complete 2025 Beginner’s Guide: Master the 5 Hottest Coding Languages and Launch Your Tech Career with Confidence.",Dr. Ethan Everhart,"Mar 11, 2025",$0.00,$23.00,$30.00,2025-03-18
1,"Ace the Data Engineering Interview: Questions and Answers for Python, SQL, Data Modeling and More",Sean Coyne,"Mar 5, 2025",$0.00,$19.99,N/A,2025-03-18
2,SQL: Learn SQL (using MySQL) in One Day and Learn It Well. SQL for Beginners with Hands-on Project. (Learn Coding Fast with Hands-On Project Book 5),Jamie Chan,"Oct 31, 2018",$0.99,$11.92,N/A,2025-03-18
3,Learn SQL and Database Management: A Beginner’s Guide to Storing and Organizing Data (Tech Management Book 2),Michael Scott,"Mar 8, 2025",$0.00,$12.08,N/A,2025-03-18
4,SQL: The Ultimate Intermediate Guide to Learning SQL Programming Step by Step (Computer Programming),Mark Reed,"Feb 17, 2025",$0.00,$19.99,N/A,2025-03-18
5,"Continue Learning SQL By Examples: One Hundred Examples of SQL Queries, Stored Procedures, and Functions for MySQL (Learn SQL Book 2)",Sergey Skudaev,"Mar 10, 2025",$0.00,$10.99,N/A,2025-03-18
6,The Art of SQL: Advanced Techniques for Database Queries,Abhishek Gautam,"Feb 28, 2025",$0.00,$10.00,N/A,2025-03-18
7,SQL Performance Optimization Cheat Sheet: Boost Query Speed | Indexing & Query Tuning Best Practices,S M,"Feb 18, 2025",$0.00,N/A,N/A,2025-03-18
8,"SQL: The Practical Guide to Master Relational Databases, SQL Syntax, and Sublanguages for Effective Database Management (Rheinwerk Computing)",Kerem Koseoglu,"Feb 26, 2025",$49.95,$40.38,N/A,2025-03-18
9,Mastering SQL Databases with LLMs: AI-Powered Querying and Optimization,Richard D Contreras,"Feb 25, 2025",$9.56,$25.67,N/A,2025-03-18


In [38]:
# Write to CSV file
df.to_csv('Amazon dynamic web scrapping.csv', index=False)